In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nanoGPT.chat import init_model as init_nanoGPT
from  nanoGPT.chat import respond as get_respond_nanoGPT
import torch
from bert_score import score
import tiktoken
from sklearn.metrics import classification_report, accuracy_score

loading GPT-2 encodings...


In [3]:
data_path = '../../data/emotion/validation/100_validation.csv'
df = pd.read_csv(data_path)

In [4]:
df.head()

,Situation,grouped_emotion,empathetic_dialogues,labels
0,Last night I heard strange noises coming from ...,afraid,In the middle of the night I heard some weird ...,Should have grabbed the gun.
1,My mom and sister threw me a baby shower when ...,excited,that was very nice of them congratulations,"Thank you! It was so nice, I had no idea it w..."
2,I just applied for a new job. After the inter...,grateful,"Oh really, do you feel like you did a great job?",I do! I'm feeling very optimistic about it
3,I loaned some money to my friend at work. Turn...,annoyed,Wow! What a jerk for him to up and leave with ...,"It was a medium amount of money but still, he ..."
4,I was out walking late last night and seen som...,NaN,Oh my god. What happened?,"Well, I started walking much faster. It looked..."


In [10]:
model_list = {
    'single_conversation': 'block_size=64/withoutemotion/singleConversation',
    'whole_conversation':'block_size=64/withoutemotion/wholeConversation',
    'single_conversation_withemotion':'block_size=64/withemotion',
    'single_conversation_withcontext': 'block_size=64/withcontext',
    'single_conversation_withGPTdata': 'block_size=64/withoutemotion/singleConversation_withGPTdata',
    'single_conversation_withGPTdata_bs256': 'block_size=256/singleConversation_withGPTdata',
}

for model_type, model_path in model_list.items():
    model_list[model_type] = init_nanoGPT(model_path)


Loading model from: ../trained-saved/block_size=64/withoutemotion/singleConversation/ckpt.pt


/Users/sofiagermer/Desktop/SOFIA/IAS/WinterSemester_24_25/ml_proj/Project-ML/src/nanoGPT/chat.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ck

number of parameters: 3.42M
Loading model from: ../trained-saved/block_size=64/withoutemotion/wholeConversation/ckpt.pt
number of parameters: 3.42M
Loading model from: ../trained-saved/block_size=64/withemotion/ckpt.pt
number of parameters: 3.42M
Loading model from: ../trained-saved/block_size=64/withcontext/ckpt.pt
number of parameters: 3.42M
Loading model from: ../trained-saved/block_size=64/withoutemotion/singleConversation_withGPTdata/ckpt.pt
number of parameters: 3.42M
Loading model from: ../trained-saved/block_size=256/singleConversation_withGPTdata/ckpt.pt
number of parameters: 3.45M


In [24]:
def get_response_from_nanoGPT(row,model):
    emotion = row['label_emotion']
    human = row['label_output']
    start = '<bot> ' + human + '<human>'
    response, model_emotion, _ = get_respond_nanoGPT(start, 1, model=model, enable_print=True)
    print("response", response)
    print("model_emotion", model_emotion)

    return response, model_emotion 

## Evaluate emotion tag generation

Evaluate the emotion tag generation from the model that was trained with it: single_conversation_withemotion.

In [21]:
df.head()

# Display the list of all columns in the DataFrame
print(df.columns.tolist())

emotion_df = df.copy()

# Remove the "Situation" column
emotion_df = emotion_df.drop(columns=["Situation"])


# Rename the column
emotion_df = emotion_df.rename(columns={"grouped_emotion": "label_emotion"})
emotion_df = emotion_df.rename(columns={"empathetic_dialogues": "input"})
emotion_df = emotion_df.rename(columns={"labels": "label_output"})


['Situation', 'grouped_emotion', 'empathetic_dialogues', 'labels']


In [22]:
emotion_df.head()

,label_emotion,input,label_output
0,afraid,In the middle of the night I heard some weird ...,Should have grabbed the gun.
1,excited,that was very nice of them congratulations,"Thank you! It was so nice, I had no idea it w..."
2,grateful,"Oh really, do you feel like you did a great job?",I do! I'm feeling very optimistic about it
3,annoyed,Wow! What a jerk for him to up and leave with ...,"It was a medium amount of money but still, he ..."
4,NaN,Oh my god. What happened?,"Well, I started walking much faster. It looked..."


In [28]:
print("model being used: ", 'single_conversation_withemotion')

def get_response_and_save(row, model):
    response, emotion = get_response_from_nanoGPT(row, model)
    return pd.Series([response, emotion])

model = model_list['single_conversation_withemotion']
emotion_df[['model_output', 'model_emotion']] = emotion_df.apply(lambda row: get_response_and_save(row, model), axis=1)

emotion_df['model_emotion'] = emotion_df['model_emotion'].str.strip() # remove the \n at the end 

model being used:  single_conversation_withemotion
output <bot> Should have grabbed the gun.<human> I saw it. I would have been so upset. <endOfText>
<emotion> disgusted
<bot> I was so disgusted!
<human> I'm sorry. I hope you found a thing. <endOfText>
<emotion> disgusted
<bot> I was so embarrassed. I was so embarrassed. I feel bad about it.
<human> Oh no, what happened? <endOfText>
<emotion> disgusted
<
Emotion:  disgusted

Robot:  I saw it. I would have been so upset. 
----Debug: Full output--- 
<bot> Should have grabbed the gun.<human> I saw it. I would have been so upset. <endOfText>
<emotion> disgusted
<bot> I was so disgusted!
<human> I'm sorry. I hope you found a thing. <endOfText>
<emotion> disgusted
<bot> I was so embarrassed. I was so embarrassed. I feel bad about it.
<human> Oh no, what happened? <endOfText>
<emotion> disgusted
<
response  I saw it. I would have been so upset. 
model_emotion  disgusted

output <bot> Thank you!  It was so nice, I had no idea it was happening.

In [26]:
# print number of rows of emotion dataframe
print("Number of rows in emotion dataframe: ", len(emotion_df))

Number of rows in emotion dataframe:  100


In [29]:
new_order = ['input', 'label_output', 'model_output', 'label_emotion', 'model_emotion']  # Replace with your desired column order
emotion_df = emotion_df[new_order]

emotion_df.head(30)

,input,label_output,model_output,label_emotion,model_emotion
0,In the middle of the night I heard some weird ...,Should have grabbed the gun.,I saw it. I would have been so upset.,afraid,disgusted
1,that was very nice of them congratulations,"Thank you! It was so nice, I had no idea it w...","I know, I'm so glad that I've done it.",excited,grateful
2,"Oh really, do you feel like you did a great job?",I do! I'm feeling very optimistic about it,I'm glad you are okay. How long ago was that?,grateful,grateful
3,Wow! What a jerk for him to up and leave with ...,"It was a medium amount of money but still, he ...","Oh, was it a bad time?",annoyed,disgusted
4,Oh my god. What happened?,"Well, I started walking much faster. It looked...",I was too scared.,NaN,afraid
5,"Yeah, it was the orthodontist but my teeth are...","I had braces growing up too, the worst part fo...",I bet that was a good time.,grateful,excited
6,That's a good idea. There's a lot of preparati...,"Yes, and it really sucks if you forget somethi...",I was able to move out.,prepared,disappointed
7,"Yesterday, I was crossing the street and the p...",O no! That's a terrible thing to have happen o...,I was so angry!,NaN,disappointed
8,It was filled with used diapers!,"Wow, that is so disgusting.",I had to walk right home.,disgusted,disgusted
9,Last christmas i got a car!,Wow.Really? What kind?,It was a new movie. I was thinking about the ...,excited,prepared


In [30]:
unique_emotions = emotion_df['label_emotion'].dropna().unique()
print("Emotions in 'label_emotion':")
for emotion in unique_emotions:
    print(emotion)

print("")
unique_emotions = emotion_df['model_emotion'].dropna().unique()
print("Emotions in 'model_emotion':")
for emotion in unique_emotions:
    print(emotion)


Emotions in 'label_emotion':
afraid
excited
grateful
annoyed
prepared
disgusted
disappointed
impressed

Emotions in 'model_emotion':
disgusted
grateful
afraid
excited
disappointed
prepared
annoyed
impressed


In [33]:
# Identify instances where the values differ
differences = emotion_df[emotion_df['label_emotion'] != emotion_df['model_emotion']]

In [35]:
def evaluate_emotion_tags(reference, candidate):
    if len(reference) != len(candidate):
        raise ValueError("The length of reference and candidate arrays must be the same.")

    # Compute accuracy
    accuracy = accuracy_score(reference, candidate)

    # Generate classification report
    class_report = classification_report(reference, candidate, output_dict=True)

    # Extract precision, recall, and F1-score (macro-average)
    precision = class_report['macro avg']['precision']
    recall = class_report['macro avg']['recall']
    f1_score = class_report['macro avg']['f1-score']

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score,
        "classification_report": class_report
    }

# Replace NaN or invalid values in reference and candidate arrays
emotion_df['label_emotion'] = emotion_df['label_emotion'].astype(str).fillna('Unknown')
emotion_df['model_emotion'] = emotion_df['model_emotion'].astype(str).fillna('Unknown')

# Reconstruct arrays after cleaning
reference = emotion_df['label_emotion'].values
candidate = emotion_df['model_emotion'].values

# Call the function again
results = evaluate_emotion_tags(reference, candidate)

# Print results
print("Evaluation Results:")
print("Accuracy:", results['accuracy'])
print("Precision:", results['precision'])
print("Recall:", results['recall'])
print("F1-Score:", results['f1_score'])

Evaluation Results:
Accuracy: 0.18
Precision: 0.14819732736399402
Recall: 0.15104656552024975
F1-Score: 0.14042327809975966


/opt/anaconda3/envs/ml/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/ml/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/ml/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
